### IS4242 Group 8 Project

### Baseline Models - Naive Bayes and PassiveAgressive

In [1]:
!pip install scikit-learn

You should consider upgrading via the 'C:\Users\edmun\Dropbox\NUS\Work\Y4S2\IS4242\is4242-group8\venv\Scripts\python.exe -m pip install --upgrade pip' command.



  Using cached scikit_learn-1.2.2-cp39-cp39-win_amd64.whl (8.4 MB)
  Using cached threadpoolctl-3.1.0-py3-none-any.whl (14 kB)
  Using cached scipy-1.10.1-cp39-cp39-win_amd64.whl (42.5 MB)


In [12]:
# importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import confusion_matrix, classification_report

import warnings
warnings.filterwarnings('ignore')

In [6]:
# Constants
RANDOM_STATE = 42
TEST_SIZE = 0.2

In [5]:
# Loading dataset
df = pd.read_csv('data/df_combined_processed_2023-03-30.csv')
df


,body,hate,privacy,sexual,impersonation,illegal,advertisement,ai
0,career career kalleron design refus collect re...,0,0,0,0,0,0,0
1,career career hotmeringue2880 hi econom gradua...,0,0,0,0,0,0,0
2,career career milenakowalska hey year finish b...,0,0,0,0,0,0,0
3,career career rstonerphd studi make employe wa...,0,0,0,0,0,1,0
4,career career temporarysection50 hi accept job...,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
41198,you muthafin lie 8220lifeask 20pearl coreyeman...,1,0,0,0,0,0,0
41199,youv gone broke wrong heart babi drove redneck...,0,0,0,0,0,0,0
41200,young buck wanna eat dat nigguh like aint fuck...,1,0,0,0,0,0,0
41201,youu got wild bitch tellin lie,1,0,0,0,0,0,0


In [18]:
# Splitting dataset into train and test

X = df['body']
y = df.drop(['body', 'privacy', 'impersonation'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE)

# Vectorizing text data

vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [36]:
def model_generator(clf_model, X_train, y_train, X_test, y_test):
    '''
    Function to generate a model and print the classification report
    
    Parameters:
    model: Model to be used for classification
    X_train: Training data
    y_train: Training labels
    X_test: Testing data
    y_test: Testing labels
    
    Returns:
    model: Trained model
    '''
    model = MultiOutputClassifier(clf_model)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    model_name = clf_model.__class__.__name__
    print(f'Classification report for {model_name} model: ')
    print(classification_report(y_test, y_pred, zero_division=0, target_names=y.columns.tolist()))

    # saving model
    with open(f'models/{model_name}_model.pkl', 'wb') as f:
        pickle.dump(model, f)

    return model

In [37]:
baseline_models = [MultinomialNB(), PassiveAggressiveClassifier()]

for model in baseline_models:
    model_generator(model, X_train, y_train, X_test, y_test)

Classification report for MultinomialNB model: 
               precision    recall  f1-score   support

         hate       0.89      0.89      0.89      3826
       sexual       0.00      0.00      0.00         0
      illegal       0.00      0.00      0.00         5
advertisement       0.00      0.00      0.00         4
           ai       0.00      0.00      0.00       228

    micro avg       0.89      0.84      0.87      4063
    macro avg       0.18      0.18      0.18      4063
 weighted avg       0.84      0.84      0.84      4063
  samples avg       0.41      0.41      0.41      4063

Classification report for PassiveAggressiveClassifier model: 
               precision    recall  f1-score   support

         hate       0.90      0.90      0.90      3826
       sexual       0.00      0.00      0.00         0
      illegal       0.00      0.00      0.00         5
advertisement       1.00      0.50      0.67         4
           ai       0.93      0.81      0.86       228

    m